In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.layers import Embedding, Masking, Concatenate, GRU, Dense, Reshape
from model import CNN_with_mask

In [2]:
np.random.seed(44)
tf.random.set_seed(44)

In [3]:
# df = pd.read_csv('test.csv')
# df = pd.read_csv('new_test.csv')
df = pd.read_csv('permutation_df.csv')

In [4]:
# rally: 1292
# Score: 891
# Errors: 1288
# 至少其中一個一定有 -> 先用 Error 來看，沒有再用 Score 來看

In [5]:
def get_rally_result(df):
    result_label = []
    ignor_game_rally_index = []
    team_col = [c for c in df.columns if 'Team' in c]
    for _, df_rally in df.groupby(['Game', 'Rally']):
        start_from = df.iloc[df_rally.index[0]][team_col].tolist()

        result = df_rally[df_rally['Score'] == 1]
        if(len(result) != 0):
            if(result.iloc[0][team_col].tolist() == start_from):
                result_label.append([0, 1])
            else:
                result_label.append([1, 0])
        else:
            result = df_rally[df_rally['Errors'] == 1]
            if(len(result) == 0):
                ignor_game_rally_index.append((df_rally.iloc[0]['Game'], df_rally.iloc[0]['Rally']))
            elif(result.iloc[0][team_col].tolist() != start_from):
                result_label.append([0, 1])
            else:
                result_label.append([1, 0])
    return tf.constant(result_label, dtype=float), ignor_game_rally_index

In [6]:
label, ignor = get_rally_result(df)

In [7]:
def get_padding_data(df, ignor):
    space_col = [c for c in df.columns if 'Space' in c]
    action_col = [c for c in df.columns if 'Action' in c]
    result_col = ['Errors', 'Score', 'Nothing']
    others_col = [c for c in df.columns if c not in space_col and c not in action_col and c not in result_col and c != 'Game' and c != 'Rally' and c != 'Team']
    team_col = [c for c in df.columns if 'Team' in c]

    rally_set = []
    rally_space_set = []
    rally_action_set = []
    for _, df_rally in df.groupby(['Game', 'Rally']):   # each rally in one game
        if((df_rally.iloc[0]['Game'], df_rally.iloc[0]['Rally']) in ignor):
            continue
        start_from = df.iloc[df_rally.index[0]][team_col].tolist()
        shot_set = []
        shot_space_set = []
        shot_action_set = []
        
        atk_sequence = []
        atk_space_sequence = []
        atk_action_sequence = []

        curr_team = start_from
        for _, shot in df_rally.iterrows():
            if(shot[team_col].tolist() != curr_team or len(atk_sequence) == 3):
                shot_set.append(atk_sequence)
                shot_space_set.append(atk_space_sequence)
                shot_action_set.append(atk_action_sequence)
                
                curr_team = shot[team_col].tolist()

                atk_sequence = []
                atk_space_sequence = []
                atk_action_sequence = []

            atk_space_sequence.append(shot[space_col])
            atk_action_sequence.append(shot[action_col])
            atk_sequence.append(shot[others_col])
        
        # the last shot
        shot_set.append(atk_sequence)
        shot_space_set.append(atk_space_sequence)
        shot_action_set.append(atk_action_sequence)

        # one rally has been finished
        shot_set = pad_sequences(shot_set, maxlen=3, padding='post')
        shot_space_set = pad_sequences(shot_space_set, maxlen=3, padding='post')
        shot_action_set = pad_sequences(shot_action_set, maxlen=3, padding='post')

        # one rally has been finished
        rally_set.append(shot_set)
        rally_space_set.append(shot_space_set)
        rally_action_set.append(shot_action_set)

    padded_rally_set = pad_sequences(rally_set, dtype=float, padding='post')
    padded_rally_space_set = pad_sequences(rally_space_set, dtype=float, padding='post')
    padded_rally_action_set = pad_sequences(rally_action_set, dtype=float, padding='post')
    
    return padded_rally_set, padded_rally_space_set, padded_rally_action_set

In [8]:
space_replace = {list(df.groupby('Space').groups.keys())[i]: i+1 for i in range(len(df.groupby('Space')))}
action_replace = {list(df.groupby('Action').groups.keys())[i]: i+1 for i in range(len(df.groupby('Action')))}

# df = pd.get_dummies(df, columns=['Team', 'No.'])
df = pd.get_dummies(df, columns=['Player'])
df = df.replace(space_replace)
df = df.replace(action_replace)

In [9]:
df

,Game,Rally,Team,Space,Action,Errors,Nothing,Score,Player_BRA_1,Player_BRA_11,...,Player_USA_18,Player_USA_19,Player_USA_2,Player_USA_20,Player_USA_22,Player_USA_4,Player_USA_5,Player_USA_6,Player_USA_8,Player_USA_9
0,7,20.0,ITA,8,15,NaN,1.0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,20.0,ITA,2,12,NaN,1.0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,20.0,ITA,2,3,NaN,NaN,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,20.0,FRA,17,13,NaN,1.0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,20.0,FRA,5,8,1.0,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7142,11,23.0,NED,3,3,NaN,1.0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
7143,11,23.0,ITA,14,13,NaN,1.0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
7144,11,23.0,ITA,6,8,NaN,1.0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
7145,11,23.0,ITA,1,11,NaN,1.0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
rally_set, rally_space_set, rally_action_set= get_padding_data(df, ignor)

# rally數, 最大回合數in one rally, 3, feature數
print(rally_set.shape)
print(rally_space_set.shape)
print(rally_action_set.shape)
print(label.shape)

(1291, 17, 3, 127)
(1291, 17, 3, 1)
(1291, 17, 3, 1)
(1291, 2)


---

In [11]:
rally_space_set = rally_space_set.squeeze()
rally_action_set = rally_action_set.squeeze()

In [12]:
rally_set_tensor = tf.convert_to_tensor(rally_set)
rally_space_set_tensor = tf.convert_to_tensor(rally_space_set)
rally_action_set_tensor = tf.convert_to_tensor(rally_action_set)
rally_result_tensor = tf.convert_to_tensor(label)

In [13]:
# rally_num = rally_set.shape[0]
rally_size = rally_set.shape[1]
shot_size = 3
feature_dim = (rally_set.shape[-1], len(df.groupby('Space'))+1, len(df.groupby('Action'))+1)
space_embed_size = 8
action_embed_size = 8
shot_embed_size = 16

In [14]:
def create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size):
    '''
    framework: 
    1. 對 space, action 做 embeding, (input, output) = (feature_dim, embed_size)
    2. concat space, action, others 成一個 embedded vector for each atk, (input) =  ([feature_dim, embed_size, embed_size])
    3. 先做 embedding
    4. CNN, filters = shot_embed_size
    5. GRU
    '''
    # each input: 三個維度, rally shot feature
    input_others = keras.Input(shape=(rally_size, shot_size, feature_dim[0]))
    input_space = keras.Input(shape=(rally_size, shot_size))
    input_action = keras.Input(shape=(rally_size, shot_size))

    

    # space & action 先做 embedding, 再和 others concat
    embed_space_layer = Embedding(input_dim=feature_dim[1], output_dim=space_embed_size, mask_zero=True, name='Space_Embedding')
    embed_action_layer = Embedding(input_dim=feature_dim[2], output_dim=action_embed_size, mask_zero=True, name='Action_Embedding')
    masking_layer = Masking(mask_value=0)   # for input_others (還沒有經過mask)

    embed_others_layer = Dense(units=36, activation='relu')

    concat_layer = Concatenate(name='Input_Concat')

    embed_shot_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=3, name='Shot_Embedding')

    cnn_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=1, name='CNN_Layer')
    gru_layer = GRU(units=16, name='GRU_Layer')
    dense_layer = Dense(units=2, activation='softmax')

    # forward
    inputs = [input_others, input_space, input_action]

    embed_space = embed_space_layer(input_space)
    embed_action = embed_action_layer(input_action)
    masked_others = masking_layer(tf.cast(input_others, tf.float32))
    embed_others = embed_others_layer(masked_others)
    embed_input = concat_layer([embed_others, embed_space, embed_action])
    embed_shot = tf.squeeze(embed_shot_layer(embed_input), axis=2)

    cnn_output = cnn_layer(embed_shot)
    gru_output = gru_layer(cnn_output)
    output = dense_layer(gru_output)
    model = keras.Model(inputs=inputs, outputs=output, name='Classification')
    return model

In [15]:
model = create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size)
model.summary()

Model: "Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 17, 3, 127)] 0                                            
__________________________________________________________________________________________________
tf.cast (TFOpLambda)            (None, 17, 3, 127)   0           input_1[0][0]                    
__________________________________________________________________________________________________
masking (Masking)               (None, 17, 3, 127)   0           tf.cast[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 17, 3)]      0                                            
_____________________________________________________________________________________

In [16]:
def split_data(others_tensor, space_tensor, action_tensor, label_tensor):
    l = label_tensor.shape[0]
    split_persentage = int(l*0.9)

    train_space = space_tensor[:split_persentage]
    train_action = action_tensor[:split_persentage]
    train_others = others_tensor[:split_persentage]
    train_label = label[:split_persentage]

    test_space = space_tensor[split_persentage:]
    test_action = action_tensor[split_persentage:]
    test_others = others_tensor[split_persentage:]
    test_label = label[split_persentage:]

    train_x = [train_others, train_space, train_action]
    train_y = train_label

    test_x = [test_others, test_space, test_action]
    test_y = test_label

    return train_x, train_y, test_x, test_y

In [17]:
train_x, train_y, test_x, test_y = split_data(rally_set_tensor, rally_space_set_tensor, rally_action_set_tensor, rally_result_tensor)

In [18]:
# regularizer = tf.keras.regularizers.l2(0.01)
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
epochs = 30
callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True, monitor='val_loss')

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [19]:
model.fit(train_x, train_y, epochs=epochs, validation_split=0.1, callbacks=[callbacks])

Epoch 1/30
33/33 [==============================] - 2s 17ms/step - loss: 0.6904 - accuracy: 0.5460 - val_loss: 0.6951 - val_accuracy: 0.5128
Epoch 2/30
33/33 [==============================] - 0s 7ms/step - loss: 0.6890 - accuracy: 0.5469 - val_loss: 0.6942 - val_accuracy: 0.5128
Epoch 3/30
33/33 [==============================] - 0s 7ms/step - loss: 0.6894 - accuracy: 0.5469 - val_loss: 0.6964 - val_accuracy: 0.5128
Epoch 4/30
33/33 [==============================] - 0s 7ms/step - loss: 0.6890 - accuracy: 0.5469 - val_loss: 0.6943 - val_accuracy: 0.5128
Epoch 5/30
33/33 [==============================] - 0s 7ms/step - loss: 0.6884 - accuracy: 0.5469 - val_loss: 0.6917 - val_accuracy: 0.5128
Epoch 6/30
33/33 [==============================] - 0s 7ms/step - loss: 0.6664 - accuracy: 0.5785 - val_loss: 0.5634 - val_accuracy: 0.7009
Epoch 7/30
33/33 [==============================] - 0s 7ms/step - loss: 0.4479 - accuracy: 0.8429 - val_loss: 0.4283 - val_accuracy: 0.8291
Epoch 8/30
33/33 [=

In [20]:
result = model.evaluate(test_x, test_y)

5/5 [==============================] - 0s 3ms/step - loss: 0.5307 - accuracy: 0.8077


In [21]:
y_pred = model.predict(test_x)

In [22]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [23]:
argmax_y_pred = np.argmax(y_pred, axis=1)
argmax_test_y = np.argmax(test_y, axis=1)

In [24]:
from sklearn.metrics import accuracy_score, roc_auc_score
from evaluate import calculate_BS, f1, show_eval_result_2class

acc_score = accuracy_score(argmax_test_y, argmax_y_pred)
f1_score = f1(argmax_test_y, argmax_y_pred)
auc_score = roc_auc_score(test_y, y_pred)
BS = calculate_BS(test_y, y_pred, 2)

In [25]:
show_eval_result_2class(acc_score, f1_score, auc_score, BS)

accuracy:    0.81
f1:          0.78
auc:         0.86
BS:          0.16


In [26]:
# model.save_weights('bestModel/sportmodel_best.h5')